# OAK apply command: Examples

Show inline help:

In [1]:
!runoak apply --help

Usage: runoak apply [OPTIONS] [COMMANDS]...

  Applies a patch to an ontology. The patch should be specified using KGCL
  syntax, see https://github.com/INCATools/kgcl

  Example:

      runoak -i cl.owl.ttl apply "rename CL:0000561 to 'amacrine neuron'"  -o
      cl.owl.ttl -O ttl

  On an obo format file:

      runoak -i simpleobo:go-edit.obo apply "rename GO:0005634 from 'nucleus'
      to 'foo'" -o go-edit-new.obo

  With URIs:

      runoak -i cl.owl.ttl apply           "rename
      <http://purl.obolibrary.org/obo/CL_0000561> from 'amacrine cell' to
      'amacrine neuron'"            -o cl.owl.ttl -O ttl


  This command is still experimental. Some things to bear in mind:

  - for some ontologies, CURIEs may not work, instead specify a full URI
  surrounded by <>s - only a subset of KGCL commands are supported by each
  backend

Options:
  -o, --output TEXT
  --changes-output TEXT         output file for KGCL changes
  --changes-input FILENAME      Path to an input changes file

## Download example file

A typical use case for the `apply` command is for applying changes to the source, aka *edit* version of an ontology.
For our purposes here we will make a copy of the go editorial file.

In [7]:
!curl -L -s https://github.com/geneontology/go-ontology/raw/master/src/ontology/go-edit.obo > input/go-edit.obo

## Create a new exact synonym

Next we will create a new change of type [NewSynonym](https://w3id.org/kgcl/NewSynonym), using KGCL syntax
on the command line.

We will first run in `--dry-run` mode:

In [3]:
!runoak -i simpleobo:input/go-edit.obo apply "create exact synonym 'compartment' for GO:0043226" --dry-run

create exact synonym 'compartment' for GO:0043226

next we will try the real deal, and save the output file:

In [4]:
!runoak --stacktrace -i simpleobo:input/go-edit.obo apply "create exact synonym 'compartment' for GO:0043226" -o output/go-edit-modified.obo

Let's try a plain unix diff (one advantage of OBO format is its easy diffability):

In [5]:
!diff -u input/go-edit.obo output/go-edit-modified.obo

--- input/go-edit.obo	2023-01-20 12:36:57.000000000 -0800
+++ output/go-edit-modified.obo	2023-01-20 12:37:07.000000000 -0800
@@ -241846,6 +241846,7 @@
 xref: NIF_Subcellular:sao1539965131
 xref: Wikipedia:Organelle
 is_a: GO:0110165 ! cellular anatomical entity
+synonym: "compartment" EXACT []
 
 [Term]
 id: GO:0043227


We can also check the command did the right thing by applying the `diff` command.

This is the reciprocal of the `apply` command, and it will generate a set of change objects in KGCL (which can then be applied....)

In [24]:
!runoak -i simpleobo:input/go-edit.obo diff -X simpleobo:output/go-edit-modified.obo -O json

[
{
  "id": "uuid:52ddc6a5-abac-40fa-9d33-cdd5eadf477d",
  "new_value": "compartment",
  "about_node": "GO:0043226",
  "@type": "NewSynonym"
}
]


## Expanding complex changes into atomic changes

Some changes represent composites of multiple smaller changes; other changes might *entail* other changes.
Some of these may be variable depending on particular ontology *workflows*.

For example, in many OBO workflows, the act of performing a [NodeObsoletion](https://w3id.org/kgcl/NodeObsoletion) might also involve:

- *renaming* the node, preceding the label with "`obsolete `"
- *rewiring* the surrounding nodes, such that the obsolete node is bypassed
- *deleting edges* such that there are no logical axioms that reference the obsoleted node

first let's try a dry run simulating what it would be like to obsolete *organelle* (GO:0043226).

First let's explore the neighborhood - we will use the `viz` command to view a random child of organelle, *non-membrane-bounded organelle* (GO:0043228)

In [23]:
!runoak -i simpleobo:input/go-edit.obo viz -p i,p GO:0043228 GO:0043226 -o output/nmbo.png

![img](output/nmbo.png)

now let's try obsoleting it, but in `--dry-run` mode, with `--expand`. (Note `--expand` is the default,
but it helps to make this explicit)

This will trigger the outputting of all expanded changes as KGCL syntax:

In [17]:
!runoak -i simpleobo:input/go-edit.obo apply --expand "obsolete GO:0043226" --dry-run

obsolete GO:0043226
rename GO:0043226 from 'organelle' to 'obsolete organelle'
create edge GO:0005929 rdfs:subClassOf GO:0110165
create edge GO:0043228 rdfs:subClassOf GO:0110165
create edge GO:0043227 rdfs:subClassOf GO:0110165
create edge GO:0043230 rdfs:subClassOf GO:0110165
create edge GO:0099572 rdfs:subClassOf GO:0110165
delete edge GO:0005929 rdfs:subClassOf GO:0043226
delete edge GO:0043228 rdfs:subClassOf GO:0043226
delete edge GO:0020004 BFO:0000050 GO:0043226
delete edge GO:0031676 BFO:0000050 GO:0043226
delete edge GO:0043227 rdfs:subClassOf GO:0043226
delete edge GO:0032420 BFO:0000050 GO:0043226
delete edge GO:0043230 rdfs:subClassOf GO:0043226
delete edge GO:0044232 BFO:0000050 GO:0043226
delete edge GO:0060091 BFO:0000050 GO:0043226
delete edge GO:0060171 BFO:0000050 GO:0043226
delete edge GO:0097591 BFO:0000050 GO:0043226
delete edge GO:0097592 BFO:0000050 GO:0043226
delete edge GO:0097593 BFO:0000050 GO:0043226
delete edge GO:0097594 BFO:0000050 GO:0043226
delete edge

in future it will be possible to visualize KGCL directly. For now, let's just visualize the output file after running in non-dry-run mode:

In [19]:
!runoak -i simpleobo:input/go-edit.obo apply --expand "obsolete GO:0043226" -o output/obsoleted-organelle.obo

In [22]:
!runoak --stacktrace -i simpleobo:output/obsoleted-organelle.obo viz -p i,p GO:0043228 GO:0043226 -o output/nmbo2.png

![img](output/nmbo2.png)